# Librerias utilizadas

In [ ]:
#r "nuget:Microsoft.Data.Analysis,0.4.0"
#r "nuget:xplot.plotly.interactive"
#r "nuget: HtmlAgilityPack, 1.11.42"
#r "nuget: ScrapySharp, 3.0.0"

using System;
using System.Collections.Generic;
using System.Globalization;
using System.IO;
using System.Linq;
using System.Threading.Tasks;
using ScrapySharp.Extensions;
using Microsoft.Data.Analysis;
using HtmlAgilityPack;
using Microsoft.AspNetCore.Html;
using Microsoft.DotNet.Interactive.Formatting;
using XPlot.Plotly;


Installed Packages HtmlAgilityPack, 1.11.42 Microsoft.Data.Analysis, 0.4.0 ScrapySharp, 3.0.0 xplot.plotly.interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

# Declaracion de la clase para almacenar los lenguajes

In [ ]:
public class Lenguaje
    {
        public string leng { get; set; }
        public int apariciones { get; set; }
        public double ratio { get; set; }
        public Lenguaje(string leng, int apariciones, double ratio)
        {
            this.leng = leng;
            this.apariciones = apariciones;
            this.ratio = ratio;
        }

        public void setRatio( double max, double min)
        {
            this.ratio = (((double)apariciones - min) / (max - min)) * 100.0;
        }

        public void imprimir()
        {
            Console.WriteLine(leng, ratio, apariciones);
        }

        public override string ToString()
        {
            return leng + ", " + ratio + ", " + apariciones;
        }
    }
    //dado un link retorna el nombre del lenguaje
    static String getLenguaje(string link)
    {
        int firstStringPosition = link.IndexOf("s/");
        return link.Substring(firstStringPosition+2);
    }

# Programa para obtener los datos de las paginas listadas

In [ ]:
            //obtener los lenguajes
            string fileName = @"./Resultados.csv";
            FileInfo fi = new FileInfo(fileName);
            string archivo2 = @"./plotData.csv";
            FileInfo fi2 = new FileInfo(archivo2);
            List<string> links = new List<string>(new string[] {
                "https://github.com/topics/matlab",
                "https://github.com/topics/perl",
                "https://github.com/topics/lua",
                "https://github.com/topics/python",
                "https://github.com/topics/c",
                "https://github.com/topics/java",
                "https://github.com/topics/cpp",
                "https://github.com/topics/csharp",
                "https://github.com/topics/visual-basic",
                "https://github.com/topics/javascript",
                "https://github.com/topics/assembly",
                "https://github.com/topics/sql",
                "https://github.com/topics/php",
                "https://github.com/topics/r",
                "https://github.com/topics/delphi",
                "https://github.com/topics/go",
                "https://github.com/topics/visual-basic-6",
                "https://github.com/topics/objective-c",
                "https://github.com/topics/swift",
                "https://github.com/topics/ruby",
            });

            try
            {
                // Si el archivo ya existe borrarlo

                if (fi.Exists)
                {
                    fi.Delete();
                }

                List<Lenguaje> lenguajes = new List<Lenguaje>();


                // Crear archivo para escribir en el lenguaje,apariciones   
                using (StreamWriter sw = fi.CreateText())
                {
                    var cont = 0;
                    HtmlWeb web = new HtmlWeb();
                    Console.WriteLine("Indx - Lenguaje, Apariciones");
                    foreach (var url in links)
                    {
                        cont++;
                        Task.Delay(20000);
                        HtmlDocument html = web.Load(url);
                        //Extrayendo las coincidencias


                        HtmlNode node = html.DocumentNode.CssSelect("[class='h3 color-fg-muted']").FirstOrDefault();

                        //Extrayendo el substring que coincide con are y public
                        int firstStringPosition = node.InnerText.IndexOf("are");
                        int secondStringPosition = node.InnerText.IndexOf("public");
                        string stringBetweenTwoStrings = node.InnerText.Substring(firstStringPosition + 16,
                            secondStringPosition - firstStringPosition - 17);
                        //Imprimir el lenguaje,apareciones
                        int apariciones = int.Parse(stringBetweenTwoStrings, NumberStyles.AllowThousands, new CultureInfo("en-au"));
                        lenguajes.Add(new Lenguaje(getLenguaje(url), apariciones, 0.0));
                        Console.WriteLine($"{cont} - {getLenguaje(url)},{apariciones}");

                        sw.WriteLine($"{getLenguaje(url)},{apariciones}");
                    }

                    //obtener el minimo y el maximo
                    int max = int.MinValue;
                    int min = int.MaxValue;
                    foreach (var lenguaje in lenguajes)
                    {
                        min = (lenguaje.apariciones < min) ? lenguaje.apariciones : min;
                        max = (lenguaje.apariciones > max) ? lenguaje.apariciones : max;
                    }

                    //settear los ratios usando los maximos y minimos
                    foreach (var lenguaje in lenguajes)
                    {
                        lenguaje.setRatio((double)max, (double)min);
                    }
                    //ordenar la lista descendentemente
                    lenguajes.Sort((x, y) => x.ratio.CompareTo(y.ratio));
                    lenguajes.Reverse();
                    Console.WriteLine();
                    Console.WriteLine("Lenguaje,Ratio,Apariciones");
                    Console.WriteLine("--------------------------");
                    for (var i = 0; i < lenguajes.Count; i++)
                    {
                        Console.WriteLine(lenguajes[i]);
                    }
                    
                    //cargar los 10 lenguajes con mayor numero de apariciones en orden descendente
                     if (fi2.Exists)
                    {
                        fi2.Delete();
                    }
                    using (StreamWriter sw2 = fi2.CreateText())
                    {
                        sw2.WriteLine("Lenguaje,Apariciones");
                        for (var i = 0; i < 10; i++)
                        {
                            sw2.WriteLine($"{lenguajes[i].leng},{lenguajes[i].apariciones}");
                        }
                    }
    
                }
            }
            catch (Exception Ex)
            {
                Console.WriteLine(Ex.ToString());
            }

Indx - Lenguaje, Apariciones
1 - matlab,9732
2 - perl,3841
3 - lua,9654
4 - python,278209
5 - c,44053
6 - java,165209
7 - cpp,45556
8 - csharp,47056
9 - visual-basic,893
10 - javascript,310144
11 - assembly,4786
12 - sql,24427
13 - php,89912
14 - r,26326
15 - delphi,1903
16 - go,41851
17 - visual-basic-6,68
18 - objective-c,5460
19 - swift,33996
20 - ruby,28310

Lenguaje,Ratio,Apariciones
--------------------------
javascript, 100, 310144
python, 89,70091203446897, 278209
java, 53,258233465343984, 165209
php, 28,974831976676686, 89912
csharp, 15,15370425315084, 47056
cpp, 14,669951882764225, 45556
c, 14,185232007636838, 44053
go, 13,475083527909288, 41851
swift, 10,941833614984715, 33996
ruby, 9,108089629639185, 28310
r, 8,468246494407824, 26326
sql, 7,855815993498368, 24427
matlab, 3,116655271610831, 9732
lua, 3,091500148350727, 9654
objective-c, 1,7389285207497516, 5460
assembly, 1,5215624556560328, 4786
perl, 1,2167984623124655, 3841
delphi, 0,5917903997729589, 1903
visual-basic, 0,

# Grafico de Barras

In [ ]:
var df1 = DataFrame.LoadCsv("./plotData.csv");
var chart = XPlot.Plotly.Chart.Plot(
    new Bar()
    {
        x = df1.Columns["Lenguaje"],
        y = df1.Columns["Apariciones"],
        marker = new XPlot.Plotly.Marker()
        {
            color = "green",
            colorscale = "Jet"
        }
    }
);

chart.Width = 500;
chart.Height = 500;
chart.WithXTitle("Lenguaje");
chart.WithYTitle("Apariciones");
chart.WithTitle("Lenguaje vs. Apariciones");
chart.Display();

<!DOCTYPE html>